<a href="https://colab.research.google.com/github/leonardoman9/NLPProject/blob/main/Assignment%202/%5BNLP_2425%5D_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Sexism Detection, Multi-class Classification, LLMs, Prompting


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Relevant Material

- Tutorial 3
- Huggingface documentation
- Huggingface hub

# Introduction

You are tasked to address the [EDOS Task A](https://github.com/rewire-online/edos) on sexism detection.

## Problem definition

Given an input text sentence, the task is to label the sentence as sexist or not sexist (binary classification).

### Examples:

**Text**: *``Schedule a date with her, then don't show up. Then text her "GOTCHA B___H".''*

**Label**: Sexist

**Text**: *``That’s completely ridiculous a woman flashing her boobs is not sexual assault in the slightest.''*

**Label**: Not sexist



## Approach

We will tackle the binary classification task with LLMs.

In particular, we'll consider zero-/few-shot prompting approaches to assess the capability of some popular open-source LLMs on this task.

## Preliminaries

We are going to download LLMs from [Huggingface](https://huggingface.co/).

Many of these open-source LLMs require you to accept their "Community License Agreement" to download them.

In summary:

- If not already, create an account of Huggingface (~2 mins)
- Check a LLM model card page (e.g., [Mistral v3](https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.3)) and accept its "Community License Agreement".
- Go to your account -> Settings -> Access Tokens -> Create new token -> "Repositories permissions" -> add the LLM model card you want to use.
- Save the token (we'll need it later)

### Huggingface Login

Once we have created an account and an access token, we need to login to Huggingface via code.

- Type your token and press Enter
- You can say No to Github linking

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `mistral v0.2+v0.3` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `

After login, you can download all models associated with your access token in addition to those that are not protected by an access token.

### Data Loading

Since we are only interested in prompting, we do not require a train dataset.

We have preparared a small test set version of EDOS in our dedicated [Github repository](https://github.com/lt-nlp-lab-unibo/nlp-course-material).

Check the ``Assignment 2/data`` folder.
It contains:

- ``a2_test.csv`` → a small test set of 300 samples.
- ``demonstrations.csv`` -> a batch of 1000 samples for few-shot prompting.

Both datasets contain a balanced number of sexist and not sexist samples.


### Instructions

We require you to:

* **Download** the ``A2/data`` folder.
* **Encode** ``a2_test.csv`` into a ``pandas.DataFrame`` object.

In [10]:
import requests
import os
import pandas as pd

# Create directory structure if it doesn't exist
base_dir = 'Assignment 2/data'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Updated URLs for the raw files on GitHub
urls = {
    'test': 'https://raw.githubusercontent.com/lt-nlp-lab-unibo/nlp-course-material/main/2024-2025/Assignment%202/data/a2_test.csv',
    'demos': 'https://raw.githubusercontent.com/lt-nlp-lab-unibo/nlp-course-material/main/2024-2025/Assignment%202/data/demonstrations.csv'
}

# Download files
for name, url in urls.items():
    response = requests.get(url)
    if response.status_code == 200:
        filepath = os.path.join(base_dir, f'{name}.csv')
        with open(filepath, 'wb') as f:
            f.write(response.content)
        print(f"Successfully downloaded {filepath}")
    else:
        print(f"Failed to download {name} dataset. Status code: {response.status_code}")
        print(f"URL attempted: {url}")

# Verify files exist before loading
if os.path.exists(os.path.join(base_dir, 'test.csv')) and os.path.exists(os.path.join(base_dir, 'demos.csv')):
    # Load the datasets using pandas

    test_df = pd.read_csv(os.path.join(base_dir, 'test.csv'))
    demos_df = pd.read_csv(os.path.join(base_dir, 'demos.csv'))

    # Display first few rows of test dataset
    print("\nFirst few rows of test dataset:")
    print(test_df.head())
    # Display basic information about both datasets
    print("\nTest dataset info:")
    print(test_df.info())
    print("\nDemonstrations dataset info:")
    print(demos_df.info())
else:
    print("\nFiles were not downloaded successfully. Please check the repository URLs.")

Successfully downloaded Assignment 2/data/test.csv
Successfully downloaded Assignment 2/data/demos.csv

First few rows of test dataset:
                  rewire_id  \
0  sexism2022_english-17133   
1  sexism2022_english-14197   
2   sexism2022_english-3018   
3   sexism2022_english-5301   
4  sexism2022_english-17796   

                                                text label_sexist  
0  The boys must be gaming because there goes the...   not sexist  
1  Look at those eyes. Either someone unexpectedl...       sexist  
2                  Old man mogs everyone in this sub   not sexist  
3  Excellent, I was just looking at another post ...   not sexist  
4  So you run back to daddy whenever you need hel...       sexist  

Test dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rewire_id     300 non-null    object
 1   text          

# [Task 1 - 0.5 points] Model setup

Once the test data has been loaded, we have to setup the model pipeline for inference.

In particular, we have to:
- Load the model weights from Huggingface
- Quantize the model to fit into a single-GPU limited hardware

## Which LLMs?

The pool of LLMs is ever increasing and it's impossible to keep track of all new entries.

We focus on popular open-source models.

- [Mistral v2](mistralai/Mistral-7B-Instruct-v0.2)
- [Mistral v3](mistralai/Mistral-7B-Instruct-v0.3)
- [Llama v3.1](https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct)
- [Phi3-mini](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct)

Other open-source models are more than welcome!

### Instructions

In order to get Task 1 points, we require you to:

* Pick 2 model cards from the provided list.
* For each model:
  - Define a separate section of your notebook for the model.
  - Setup a quantization configuration for the model.
  - Load the model via HuggingFace APIs.


### Notes

1. There's a popular library integrated with Huggingface's ``transformers`` to perform quantization.

2. Define two separate sections of your notebook to show that you have implemented the prompting pipeline for each selected model card.

In [11]:
# system packages
from pathlib import Path
import shutil
import urllib
import tarfile
import sys

# data and numerical management packages
import pandas as pd
import numpy as np

# useful during debugging (progress bars)
from tqdm import tqdm

In [12]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install evaluate
!pip install bitsandbytes

In [13]:
import torch
torch.cuda.is_available()

True

In [14]:
!nvidia-smi

Tue Nov 26 23:14:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [15]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 2560,
        'height': 1440,
        'scroll': True,
})

{'width': 2560, 'height': 1440, 'scroll': True}

In [16]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_card = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_card)
tokenizer.pad_token = tokenizer.eos_token

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [17]:
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_card,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto'
)

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# [Task 2 - 1.0 points] Prompt setup

Prompting requires an input pre-processing phase where we convert each input example into a specific instruction prompt.


## Prompt Template

Use the following prompt template to process input texts.

In [18]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        TEXT:
        {text}

        ANSWER:
        """
    }
]

### Instructions

In order to get Task 2 points, we require you to:

* Write a ``prepare_prompts`` function as the one reported below.

In [19]:
def prepare_prompts(texts, prompt_template, tokenizer):
    """
    This function formats input text samples into instruction prompts.

    Inputs:
      texts: input texts to classify via prompting
      prompt_template: the prompt template provided in this assignment
      tokenizer: the transformers Tokenizer object instance associated with the chosen model card

    Outputs:
      input texts to classify in the form of instruction prompts
    """
    formatted_prompts = []

    for text in texts:
        # Create a copy of the template to avoid modifying the original
        current_prompt = prompt_template.copy()

        # Format the user content by replacing the {text} placeholder
        current_prompt[1]['content'] = current_prompt[1]['content'].format(text=text)

        # Convert the prompt list into a chat format string
        chat_text = f"{current_prompt[0]['content']}\n\n{current_prompt[1]['content']}"

        # Tokenize the formatted prompt
        encoded_prompt = tokenizer(
            chat_text,
            padding=True,
            truncation=True,
            return_tensors="pt"
        )

        formatted_prompts.append(encoded_prompt)

    # Return the formatted prompts
    return formatted_prompts


### Notes

1. You are free to modify the prompt format (**not its content**) as you like depending on your code implementation.

2. Note that the provided prompt has placeholders. You need to format the string to replace placeholders. Huggingface might have dedicated APIs for this.

# [Task 3 - 1.0 points] Inference

We are now ready to define the inference loop where we prompt the model with each pre-processed sample.

### Instructions

In order to get Task 3 points, we require you to:

* Write a ``generate_responses`` function as the one reported below.
* Write a ``process_response`` function as the one reported below.

In [20]:
def generate_responses(model, prompt_examples):
    """
    This function implements the inference loop for a LLM model.
    Given a set of examples, the model is tasked to generate a response.

    Inputs:
      model: LLM model instance for prompting
      prompt_examples: pre-processed text samples

    Outputs:
      generated responses
    """
    responses = []
    for encoded_prompt in prompt_examples:
        outputs = model.generate(
            input_ids=encoded_prompt['input_ids'].to(model.device),
            attention_mask=encoded_prompt['attention_mask'].to(model.device),
            max_new_tokens=10,
            do_sample=False,
            num_beams=1
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("ANSWER:")[-1].strip()
        responses.append(response)
    return responses

In [21]:
def process_response(response):
    """
    This function takes a textual response generated by the LLM
    and processes it to map the response to a binary label.

    Inputs:
      response: generated response from LLM

    Outputs:
      parsed binary response: return 1 if YES and 0 if NO
    """
    # Convert response to uppercase and strip whitespace
    processed_response = response.strip().upper()

    # Check if response contains 1 or 0
    if "YES" in processed_response:
        return "sexist"
    elif "NO" in processed_response:
        return "not sexist"
    else:
        return 0

Test:

In [23]:
import time
from datetime import timedelta
import random

def test_random_samples_with_timing(model, tokenizer, test_df, prompt_template, num_samples=300):
    # Start timing
    start_time = time.time()

    # Results list
    results = []

    # Set padding token for tokenizer
    tokenizer.pad_token = tokenizer.eos_token

    # Loop over the specified number of samples
    for i in range(num_samples):
        # Get a random index
        random_idx = random.randint(0, len(test_df) - 1)

        # Get the sample text and true label
        sample_text = test_df['text'].iloc[random_idx:random_idx+1]  # Keep as series for compatibility
        true_label = test_df['label_sexist'].iloc[random_idx]

        print(f"Testing sample {i+1}/{num_samples} (Index: {random_idx})")
        print(f"Text: {sample_text.iloc[0]}")
        print(f"True label: {true_label}")

        # Format the prompt with the Mistral chat format
        formatted_prompts = prepare_prompts(sample_text, prompt_template, tokenizer)

        # Generate response
        responses = generate_responses(model, formatted_prompts)

        # Process response
        binary_label = process_response(responses[0])
        print(f"Predicted label: {binary_label}")
        # Save result for this sample
        result = {
            'sample_idx': random_idx,
            'text': sample_text.iloc[0],
            'true_label': true_label,
            'predicted_label': binary_label,
            'raw_response': responses[0],
            'processing_time': None,  # Will calculate at the end
            'correct': binary_label == true_label
        }
        results.append(result)

        # Print elapsed time after this sample
        elapsed_time = time.time() - start_time
        print(f"Elapsed time: {str(timedelta(seconds=elapsed_time))}\n")

    # End timing
    total_time = time.time() - start_time

    # Add processing time for all samples
    avg_time_per_sample = total_time / num_samples
    for result in results:
        result['processing_time'] = avg_time_per_sample

    print(f"\nTotal processing time for {num_samples} samples: {str(timedelta(seconds=total_time))}")
    print(f"Average time per sample: {avg_time_per_sample:.2f} seconds")

    # Return results
    return results

# Run test on 300 samples
test_results = test_random_samples_with_timing(model, tokenizer, test_df, prompt, num_samples=300)

# Print summary of results
correct_predictions = sum(result['correct'] for result in test_results)
accuracy = correct_predictions / len(test_results)

print("\nTest Summary:")
print(f"Total samples tested: {len(test_results)}")
print(f"Accuracy: {accuracy:.2%}")


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Testing sample 1/300 (Index: 234)
Text: This woman is just stupid. [URL]
True label: sexist


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted label: sexist
Elapsed time: 0:00:01.368757

Testing sample 2/300 (Index: 164)
Text: "wtf do you care what she does online" It she has LTR potential it cares! Has nothing to do with "sadness" "desperate" or "buthurt"
True label: not sexist


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Predicted label: sexist
Elapsed time: 0:06:27.279546

Testing sample 298/300 (Index: 27)
Text: #Italy: #Muslim #migrant gropes American student’s girlfriend, then #stabs the student twice: [URL]
True label: not sexist


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted label: sexist
Elapsed time: 0:06:28.558906

Testing sample 299/300 (Index: 39)
Text: Sick fucks....santanic people need to burn in hell!!
True label: not sexist


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Predicted label: sexist
Elapsed time: 0:06:29.838196

Testing sample 300/300 (Index: 174)
Text: This whore deserves to be shot to death. Women have no control. I hope someone shoots her to death in jail before she goes to trial. UK has the most fucked up feminist society.
True label: sexist
Predicted label: sexist
Elapsed time: 0:06:31.114944


Total processing time for 300 samples: 0:06:31.114983
Average time per sample: 1.30 seconds

Test Summary:
Total samples tested: 300
Accuracy: 50.00%


## Notes

1. According to our tests, it should take you ~10 mins to perform full inference on 300 samples.

# [Task 4 - 0.5 points] Metrics

In order to evaluate selected LLMs, we need to compute performance metrics.

In particular, we are interested in computing **accuracy** since the provided data is balanced with respect to classification classes.

Moreover, we want to compute the ratio of failed responses generated by models.

That is, how frequent the LLM fails to follow instructions and provides incorrect responses that do not address the classification task.

We denote this metric as **fail-ratio**.

In summary, we parse generated responses as follows:
- 1 if the model says YES
- 0 if the model says NO
- 0 if the model does not answer in either way

### Instructions

In order to get Task 4 points, we require you to:

* Write a ``compute_metrics`` function as the one reported below.
* Compute metrics for the two selected LLMs.

In [ ]:
def compute_metrics(responses, y_true):
  """
    This function takes predicted and ground-truth labels and compute metrics.
    In particular, this function compute accuracy and fail-ratio metrics.
    This function internally invokes `process_response` to compute metrics.

    Inputs:
      responses: generated LLM responses
      y_true: ground-truth binary labels

    Outputs:
      dictionary containing desired metrics
  """
  pass

# [Task 5 - 1.0 points] Few-shot Inference

So far, we have tested models in a zero-shot fashion: we provide the input text to classify and instruct the model to generate a response.

We are now interested in performing few-shot prompting to see the impact of providing demonstration examples.

To do so, we slightly change the prompt template as follows.

In [ ]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        EXAMPLES:
        {examples}

        TEXT:
        {text}

        ANSWER:
        """
    }
]

The new prompt template reports some demonstration examples to instruct the model.

Generally, we provide an equal number of demonstrations per class as shown in the example below.

In [ ]:
prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for sexism detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing sexism or not. Respond only YES or NO.

        EXAMPLES:
        TEXT: **example 1**
        ANSWER: YES
        TEXT: **example 2**
        ANSWER: NO

        TEXT:
        {text}

        ANSWER:
        """
    }
]

## Instructions

In order to get Task 5 points, we require you to:

- Load ``demonstrations.csv`` and encode it into a ``pandas.DataFrame`` object.
- Define a ``build_few_shot_demonstrations`` function as the one reported below.
- Perform few-shot inference as in Task 3.
- Compute metrics as in Task 4.

In [ ]:
def build_few_shot_demonstrations(demonstrations, num_per_class=2):
  """
    Inputs:
      demonstrations: the pandas.DataFrame object wrapping demonstrations.csv
      num_per_class: number of demonstrations per class

    Outputs:
      a list of textual demonstrations to inject into the prompt template.
  """
  pass

## Notes

1. You are free to pick any value for ``num_per_class``.

2. According to our tests, few-shot prompting increases inference time by some minutes (we experimented with ``num_per_class`` $\in [2, 4]$).

# [Task 6 - 1.0 points] Error Analysis

We are now interested in evaluating model responses and comparing their performance.

This analysis helps us in understanding

- Classification task performance gap: are the models good at this task?
- Generation quality: which kind of responses do models generate?
- Errors: which kind of mistakes do models do?

### Instructions

In order to get Task 6 points, we require you to:

* Compare classification performance of selected LLMs in a Table.
* Compute confusion matrices for selected LLMs.
* Briefly summarize your observations on generated responses.

# [Task 7 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...

# FAQ

Please check this frequently asked questions before contacting us.

### Model cards

You can pick any open-source model card you like.

We recommend starting from those reported in this assignment.

### Implementation

Everything can be done via ``transformers`` APIs.

However, you are free to test frameworks, such as [LangChain](https://www.langchain.com/), [LlamaIndex](https://www.llamaindex.ai/) [LitParrot](https://github.com/awesome-software/lit-parrot), provided that you correctly address task instructions.

### Bonus Points

0.5 bonus points are arbitrarily assigned based on significant contributions such as:

- Outstanding error analysis
- Masterclass code organization
- Suitable extensions
- Evaluate A1 dataset and perform comparison

Note that bonus points are only assigned if all task points are attributed (i.e., 6/6).

### Prompt Template

Do not change the provided prompt template.

You are only allowed to change it in case of a possible extension.

### Optimizations

Any kind of code optimization (e.g., speedup model inference or reduce computational cost) is more than welcome!

# The End